In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pylab as plt
sp.init_printing()

In [ ]:
w, f_c = sp.symbols(r'w, f_c', real=True)
B = sp.symbols(r'B', nonnegative=True)
n, d_s = sp.symbols(r'n, d_s', nonnegative=True)
s = sp.Symbol('s', nonnegative = True)
E = sp.Symbol(r'E', nonnegative=True)
p, P = sp.symbols(r'p, P', nonnegative=True)
tau = sp.symbols(r'tau', nonnegative=True)
sig_y = sp.symbols('sigma_y', positive=True)
A = sp.Symbol(r'A', nonnegative = True)

In [ ]:
params = (B, d_s, n, f_c, tau, E, sig_y)
(s,)+params

In [ ]:
param_values = dict(
    B=250,
    d_s=28,
    n=2,
    f_c=33.3,
    tau=0.4,
    E=210000,
    sigma_y=713,
    )

In [ ]:
p = sp.pi * d_s
A = sp.pi * d_s ** 2 / 4
A

## Crack bridging action

Check if this is slip or opening actually, the value $s = w / 2$ should be used here.

In [ ]:
Pw_cb = sp.sqrt(2 * w/2 * tau * E * A * p)
P_max = A * sig_y
w_argmax = sp.solve(P_max - Pw_cb, w)[0]
w_argmax

In [ ]:
Pw_cb_y = sp.Piecewise(
    (Pw_cb, w < w_argmax),
    (P_max, w >= w_argmax))

In [ ]:
Pw_cb_y

In [ ]:
get_Pw_cb_y = sp.lambdify((w,)+params+('**kw',), Pw_cb_y, 'numpy')

In [ ]:
_, (ax1) = plt.subplots(1, 1, figsize=(10,5))
w_data = np.linspace(0, 0.5, 100)
Pw_cb_y_val = get_Pw_cb_y(w_data, **param_values)
ax1.plot(w_data, Pw_cb_y_val / 1000, color='blue', lw=2);

## Dowel action

In [ ]:
b_n = B - n * d_s
V_cr = 1.64 * b_n * d_s * f_c ** sp.Rational(1,3)
# V_cr = 1.35 * b_n * d_s * f_c ** sp.Rational(1,3)
V_da_1 = V_cr * (s / 0.05) * (2 - (s / 0.05))
V_da_2 = V_cr * ((2.55 - s) / 2.5)
V_da = sp.Piecewise(
        (V_da_1, s <= 0.05),
        (V_da_2, s > 0.05))

### Verify the shape of the law

In [ ]:
get_V_da_V_cr = sp.lambdify(s, sp.simplify(V_da / V_cr))
fig, ax = plt.subplots(1,1)
s_range = np.linspace(0, 0.15, 100)
ax.plot(s_range, get_V_da_V_cr(s_range))

In [ ]:
get_V_da_val = sp.lambdify((s,)+params+('**kw',), V_da, 'numpy')

In [ ]:
_, (ax1) = plt.subplots(1, 1, figsize=(10,5))
s_data = np.linspace(0, 1, 100)
V_da_val = get_V_da_val(s_data, **param_values)
ax1.plot(s_data, V_da_val / 1000, color='blue', lw=2);